In [ ]:
# Paso 1: Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Paso 2: Cargar dataset
# Convertir a DataFrame
df = pd.read_csv("leads_marketing.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID_Lead               300 non-null    object 
 1   Fuente_Origen         300 non-null    object 
 2   Tiempo_en_Sitio_min   286 non-null    float64
 3   Visitas_Totales       300 non-null    int64  
 4   Dias_Ultimo_Contacto  300 non-null    int64  
 5   Cargo                 300 non-null    object 
 6   Sector                300 non-null    object 
 7   Convertido            300 non-null    int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 18.9+ KB


In [ ]:
# Calcular la mediana de 'Tiempo_en_Sitio_min' del DataFrame original 'df'
# Esto asegura que estamos trabajando con el estado actual de df
median_tiempo_df = df['Tiempo_en_Sitio_min'].median()

print(f"El valor atípico a reemplazar en 'df' es: 500.0 minutos")
print(f"La mediana actual de 'Tiempo_en_Sitio_min' en 'df' es: {median_tiempo_df:.2f} minutos")

# Contar cuántas veces aparece el valor 500 en 'df' antes del reemplazo
num_500_before_df = (df['Tiempo_en_Sitio_min'] == 500).sum()
print(f"Número de veces que el valor 500 aparece en 'df' antes del reemplazo: {num_500_before_df}")

# Reemplazar todas las ocurrencias del valor 500 con la mediana calculada en el DataFrame 'df'
df.loc[df['Tiempo_en_Sitio_min'] == 500, 'Tiempo_en_Sitio_min'] = median_tiempo_df

# Contar cuántas veces aparece el valor 500 en 'df' después del reemplazo (debería ser 0)
num_500_after_df = (df['Tiempo_en_Sitio_min'] == 500).sum()
print(f"Número de veces que el valor 500 aparece en 'df' después del reemplazo: {num_500_after_df}")

# Mostrar el nuevo valor máximo de la columna para confirmar el cambio en 'df'
new_max_value_df = df['Tiempo_en_Sitio_min'].max()
print(f"Nuevo valor máximo de 'Tiempo_en_Sitio_min' en 'df' después del reemplazo: {new_max_value_df:.2f} minutos")

El valor atípico a reemplazar en 'df' es: 500.0 minutos
La mediana actual de 'Tiempo_en_Sitio_min' en 'df' es: 32.25 minutos
Número de veces que el valor 500 aparece en 'df' antes del reemplazo: 1
Número de veces que el valor 500 aparece en 'df' después del reemplazo: 0
Nuevo valor máximo de 'Tiempo_en_Sitio_min' en 'df' después del reemplazo: 60.00 minutos


In [ ]:
display(df.describe())

,Tiempo_en_Sitio_min,Visitas_Totales,Dias_Ultimo_Contacto,Convertido
count,286.000000,300.000000,300.000000,300.00000
mean,31.302622,9.616667,182.080000,0.46000
std,17.825487,5.400643,104.094793,0.49923
min,1.100000,1.000000,2.000000,0.00000
25%,17.050000,5.000000,96.000000,0.00000
50%,32.175000,9.000000,188.000000,0.00000
75%,46.650000,14.000000,270.500000,1.00000
max,60.000000,19.000000,364.000000,1.00000


In [ ]:
df.head()

,ID_Lead,Fuente_Origen,Tiempo_en_Sitio_min,Visitas_Totales,Dias_Ultimo_Contacto,Cargo,Sector,Convertido
0,LEAD-0001,Google Ads,32.25,5,327,Gerente,Tecnología,0
1,LEAD-0002,Orgánico,32.10,18,257,Gerente,Salud,0
2,LEAD-0003,Linkedin,32.70,10,224,Becario,Finanzas,1
3,LEAD-0004,Referido,38.40,6,141,Analista,Retail,1
4,LEAD-0005,Facebook,43.70,1,215,Analista,Finanzas,0


In [ ]:
# Seleccion de datos: Asegurarse de que X y y se crean después de limpiar df
X = df.drop('Convertido', axis=1)
y = df['Convertido']
feature_names = X.columns

In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.isnull().sum()

,0
ID_Lead,0
Fuente_Origen,0
Tiempo_en_Sitio_min,14
Visitas_Totales,0
Dias_Ultimo_Contacto,0
Cargo,0
Sector,0
Convertido,0


In [ ]:
df.shape

(300, 8)

In [ ]:
df[df.isnull().any(axis=1)]

,ID_Lead,Fuente_Origen,Tiempo_en_Sitio_min,Visitas_Totales,Dias_Ultimo_Contacto,Cargo,Sector,Convertido
17,LEAD-0018,google ads,NaN,16,175,Gerente,Salud,0
41,LEAD-0042,Email,NaN,3,323,Analista,Retail,0
71,LEAD-0072,Facebook,NaN,6,112,Otro,Salud,0
78,LEAD-0079,google ads,NaN,6,197,Gerente,Retail,1
121,LEAD-0122,Orgánico,NaN,1,293,Analista,Tecnología,0
129,LEAD-0130,Email,NaN,13,41,Becario,Tecnología,0
138,LEAD-0139,Google Ads,NaN,15,342,Director,Educación,0
156,LEAD-0157,Linkedin,NaN,9,363,Otro,Retail,0
181,LEAD-0182,Facebook,NaN,2,289,Gerente,Finanzas,0
205,LEAD-0206,Facebook,NaN,2,108,Gerente,Retail,0


In [ ]:
df['Tiempo_en_Sitio_min'].fillna(df['Tiempo_en_Sitio_min'].median(), inplace=True)

/tmp/ipython-input-2035627036.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Tiempo_en_Sitio_min'].fillna(df['Tiempo_en_Sitio_min'].median(), inplace=True)


In [ ]:
df.isnull().sum()

,0
ID_Lead,0
Fuente_Origen,0
Tiempo_en_Sitio_min,0
Visitas_Totales,0
Dias_Ultimo_Contacto,0
Cargo,0
Sector,0
Convertido,0


### Paso 1: Preparar los datos para el modelo de Random Forest

Reutilizamos las variables predictoras (`Tiempo_en_Sitio_min`, `Visitas_Totales`, `Dias_Ultimo_Contacto`) y la variable objetivo (`Convertido`). Escalamos las características numéricas para asegurar la consistencia, aunque Random Forest es menos sensible a la escala que otros modelos.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Variables predictoras (features)
features_rf = ['Tiempo_en_Sitio_min', 'Visitas_Totales', 'Dias_Ultimo_Contacto']
X_rf = df[features_rf]

# Variable objetivo (target)
y_rf = df['Convertido']

# Escalar las características numéricas
scaler_rf = MinMaxScaler()
X_rf_scaled = scaler_rf.fit_transform(X_rf)
X_rf_scaled_df = pd.DataFrame(X_rf_scaled, columns=features_rf)

print("Primeras 5 filas de las características escaladas para Random Forest:")
display(X_rf_scaled_df.head())

Primeras 5 filas de las características escaladas para Random Forest:


,Tiempo_en_Sitio_min,Visitas_Totales,Dias_Ultimo_Contacto
0,0.528862,0.222222,0.897790
1,0.526316,0.944444,0.704420
2,0.536503,0.500000,0.613260
3,0.633277,0.277778,0.383978
4,0.723260,0.000000,0.588398


### Paso 2: Dividir el dataset en conjuntos de entrenamiento y prueba

Dividimos los datos para entrenar el modelo y luego evaluar su capacidad de generalización.

In [ ]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf_scaled_df, y_rf, test_size=0.3, random_state=42)

print(f"Tamaño del conjunto de entrenamiento (X_train_rf): {X_train_rf.shape}")
print(f"Tamaño del conjunto de prueba (X_test_rf): {X_test_rf.shape}")

Tamaño del conjunto de entrenamiento (X_train_rf): (210, 3)
Tamaño del conjunto de prueba (X_test_rf): (90, 3)


### Paso 3: Entrenar el modelo de Random Forest Classifier

Configuramos y entrenamos el modelo de Random Forest.

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # n_estimators es el número de árboles en el bosque
rf_model.fit(X_train_rf, y_train_rf)

print("Modelo de Random Forest Classifier entrenado exitosamente.")

Modelo de Random Forest Classifier entrenado exitosamente.


### Paso 4: Evaluar el rendimiento del modelo

Generamos predicciones y analizamos métricas como el reporte de clasificación y la matriz de confusión para entender qué tan bien se desempeña el modelo.

In [ ]:
y_pred_rf = rf_model.predict(X_test_rf)

print("\nReporte de Clasificación para Random Forest:")
print(classification_report(y_test_rf, y_pred_rf))

print("\nMatriz de Confusión para Random Forest:")
print(confusion_matrix(y_test_rf, y_pred_rf))


Reporte de Clasificación para Random Forest:
              precision    recall  f1-score   support

           0       0.61      0.69      0.65        45
           1       0.64      0.56      0.60        45

    accuracy                           0.62        90
   macro avg       0.62      0.62      0.62        90
weighted avg       0.62      0.62      0.62        90


Matriz de Confusión para Random Forest:
[[31 14]
 [20 25]]


### Paso 5: Realizar una predicción interactiva con el modelo de Random Forest

Ingresa los valores solicitados para obtener una predicción de conversión utilizando el modelo de Random Forest.

In [ ]:
def predict_conversion_rf(tiempo_en_sitio, visitas_totales, dias_ultimo_contacto):
    # Crear un DataFrame con los valores de entrada
    input_data_rf = pd.DataFrame([[tiempo_en_sitio, visitas_totales, dias_ultimo_contacto]],
                                 columns=features_rf)

    # Escalar los datos de entrada usando el mismo scaler_rf entrenado
    input_scaled_rf = scaler_rf.transform(input_data_rf)

    # Realizar la predicción
    prediction_rf = rf_model.predict(input_scaled_rf)
    prediction_proba_rf = rf_model.predict_proba(input_scaled_rf)[:, 1]

    if prediction_rf[0] == 1:
        print(f"El lead se predice como: Convertido (con probabilidad de {prediction_proba_rf[0]:.2f})")
    else:
        print(f"El lead se predice como: No Convertido (con probabilidad de {1 - prediction_proba_rf[0]:.2f})")

# Solicitar valores al usuario
tiempo_en_sitio_input = float(input("Ingrese el 'Tiempo_en_Sitio_min': "))
visitas_totales_input = int(input("Ingrese las 'Visitas_Totales': "))
dias_ultimo_contacto_input = int(input("Ingrese los 'Dias_Ultimo_Contacto': "))

# Realizar la predicción
predict_conversion_rf(tiempo_en_sitio_input, visitas_totales_input, dias_ultimo_contacto_input)

Ingrese el 'Tiempo_en_Sitio_min': 30
Ingrese las 'Visitas_Totales': 10
Ingrese los 'Dias_Ultimo_Contacto': 90
El lead se predice como: Convertido (con probabilidad de 0.54)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
